In [1]:
# load libraies

%run py_libraries.py

/Users/4476224/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.13.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# loading utility files

from utility.sv_fig import savefig
from utility.make_cm import make_confusion_matrix
from utility.get_g_result import get_gamma_results
from utility.plt_result import plot_results

In [3]:
def savefig(filename, crop = True):
    plt.savefig('{}.pdf'.format(filename))

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

# load data
X_train = pd.read_csv('data/norm_X_train_NCa_Ca.csv')
X_test = pd.read_csv('data/norm_X_test_NCa_Ca.csv')
y_train = pd.read_csv('data/y_train_NCa_Ca.csv')
y_test = pd.read_csv('data/y_test_NCa_Ca.csv')


print(X_train.shape)

(91, 37)


In [5]:
# Print the column names as a list
print(X_train.columns.tolist())

['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']


In [6]:
Xy_NCa_Ca = pd.concat([X_train, y_train], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy import stats

col_to_scale = ['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 
                'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 
                'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG' ]

def midas_impl(missing, col_to_scale, n_epoch):
    missing_cp = missing.copy() # make copy of missing df
    
    # scaler = MinMaxScaler().fit(missing_cp[col_to_scale]) # scale data
    scaler = StandardScaler().fit(missing_cp[col_to_scale]) # scale data
    missing_sc = scaler.transform(missing_cp[col_to_scale]) # this is an array, convert to df

    missing_df = pd.DataFrame({'ENA.78':missing_sc[:,0],
                               'IFN.y':missing_sc[:,1],
                               'IL.10':missing_sc[:,2],
                               'IL.6':missing_sc[:,3],
                               'IL.8':missing_sc[:,4],
                               'MCP.1':missing_sc[:,5],
                               'MDC':missing_sc[:,6],
                               'MIP.1a':missing_sc[:,7],
                               'TNF.a':missing_sc[:,8],
                               'C.peptide':missing_sc[:,9],
                               'G.CSF':missing_sc[:,10],
                               'IL.22':missing_sc[:,11],
                               'Insulin':missing_sc[:,12],
                               'Leptin':missing_sc[:,13],
                               'MIP.3a':missing_sc[:,14],
                               'GRO.a':missing_sc[:,15],
                               'HGF':missing_sc[:,16],
                               'MMP.2':missing_sc[:,17],
                               'Adiponectin':missing_sc[:,18],
                               'CRP':missing_sc[:,19],
                               'GDF.15':missing_sc[:,20],
                               'TIMP.1':missing_sc[:,21],
                               'TGF.B2':missing_sc[:,22],
                               'TGF.B1':missing_sc[:,23],
                               'PPAR.y':missing_sc[:,24],
                               'HIF.1a':missing_sc[:,25],
                               'Laminin':missing_sc[:,26],
                               'HbA1c':missing_sc[:,27],
                               'CA19.9':missing_sc[:,28],
                               'Glucose':missing_sc[:,29],
                               'HDL':missing_sc[:,30],
                               'CCK':missing_sc[:,31],
                               'LDL':missing_sc[:,32],
                               'Triglyceride':missing_sc[:,33],
                               'Albumin':missing_sc[:,34],
                               'Lumican':missing_sc[:,35],
                               'ZAG':missing_sc[:,36],
                               })
    
    # Extract cat columns from missing
    ext_col1 = missing_cp['CACHEXSTAGE0VIG']
    

    # Add the extracted cat columns to missing_df
    missing_df = pd.concat([missing_df, ext_col1.rename('CACHEXSTAGE0VIG')], axis=1)
    

    missing_df.to_csv('missing_NCa_Ca.csv')
    data_0 = pd.read_csv('missing_NCa_Ca.csv')
    data_0.columns.str.strip()

    categorical = ['CACHEXSTAGE0VIG']
    
    data_cat, cat_cols_list = md.cat_conv(data_0[categorical])

    data_0.drop(categorical, axis = 1, inplace = True)
    constructor_list = [data_0]
    constructor_list.append(data_cat)
    data_in = pd.concat(constructor_list, axis=1)

    na_loc = data_in.isnull()
    data_in[na_loc] = np.nan

    # imputer = md.Midas(layer_structure = [128,128], vae_layer = True, seed = 89, input_drop = 0.75, learn_rate = 0.0001)
    imputer = md.Midas(layer_structure = [128,128], vae_layer = False, seed = 42, input_drop = 0.75, learn_rate = 0.001)
    imputer.build_model(data_in, softmax_columns = cat_cols_list)
    imputer.train_model(training_epochs = n_epoch)

    # do not update M
    imputations = imputer.generate_samples(m=20).output_list

    flat_cats = [cat for variable in cat_cols_list for cat in variable]
    for i in range(len(imputations)):
        tmp_cat = [imputations[i][x].idxmax(axis=1) for x in cat_cols_list]
        cat_df = pd.DataFrame({categorical[i]:tmp_cat[i] for i in range(len(categorical))})
        imputations[i] = pd.concat([imputations[i], cat_df], axis = 1).drop(flat_cats, axis = 1)

    mapping_ca = {'CACHEXSTAGE0VIG_0.0': 0,'CACHEXSTAGE0VIG_1.0': 1}

    imputations[0] = imputations[0].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[1] = imputations[1].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[2] = imputations[2].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[3] = imputations[3].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[4] = imputations[4].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[5] = imputations[5].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[6] = imputations[6].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[7] = imputations[7].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[8] = imputations[8].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[9] = imputations[9].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[10] = imputations[10].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[11] = imputations[11].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[12] = imputations[12].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[13] = imputations[13].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[14] = imputations[14].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[15] = imputations[15].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[16] = imputations[16].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[17] = imputations[17].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[18] = imputations[18].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[19] = imputations[19].replace({'CACHEXSTAGE0VIG': mapping_ca})

    # rescale imputations
    imputations[0][col_to_scale] = scaler.inverse_transform(imputations[0][col_to_scale])
    imputations[1][col_to_scale] = scaler.inverse_transform(imputations[1][col_to_scale])
    imputations[2][col_to_scale] = scaler.inverse_transform(imputations[2][col_to_scale])
    imputations[3][col_to_scale] = scaler.inverse_transform(imputations[3][col_to_scale])
    imputations[4][col_to_scale] = scaler.inverse_transform(imputations[4][col_to_scale])
    imputations[5][col_to_scale] = scaler.inverse_transform(imputations[5][col_to_scale])
    imputations[6][col_to_scale] = scaler.inverse_transform(imputations[6][col_to_scale])
    imputations[7][col_to_scale] = scaler.inverse_transform(imputations[7][col_to_scale])
    imputations[8][col_to_scale] = scaler.inverse_transform(imputations[8][col_to_scale])
    imputations[9][col_to_scale] = scaler.inverse_transform(imputations[9][col_to_scale])
    imputations[10][col_to_scale] = scaler.inverse_transform(imputations[10][col_to_scale])
    imputations[11][col_to_scale] = scaler.inverse_transform(imputations[11][col_to_scale])
    imputations[12][col_to_scale] = scaler.inverse_transform(imputations[12][col_to_scale])
    imputations[13][col_to_scale] = scaler.inverse_transform(imputations[13][col_to_scale])
    imputations[14][col_to_scale] = scaler.inverse_transform(imputations[14][col_to_scale])
    imputations[15][col_to_scale] = scaler.inverse_transform(imputations[15][col_to_scale])
    imputations[16][col_to_scale] = scaler.inverse_transform(imputations[16][col_to_scale])
    imputations[17][col_to_scale] = scaler.inverse_transform(imputations[17][col_to_scale])
    imputations[18][col_to_scale] = scaler.inverse_transform(imputations[18][col_to_scale])
    imputations[19][col_to_scale] = scaler.inverse_transform(imputations[19][col_to_scale])

    n=0
    for i in imputations:
        file_out_csv = "midas_NCa_Ca/midas_NCa_Ca_" + str(n) + ".csv"
        i.to_csv(file_out_csv, index=False)
        n += 1

    df0 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_0.csv'); df0 = df0.iloc[:,1:]
    df1 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_1.csv'); df1 = df1.iloc[:,1:]
    df2 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_2.csv'); df2 = df2.iloc[:,1:]
    df3 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_3.csv'); df3 = df3.iloc[:,1:]
    df4 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_4.csv'); df4 = df4.iloc[:,1:]
    df5 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_5.csv'); df5 = df5.iloc[:,1:]
    df6 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_6.csv'); df6 = df6.iloc[:,1:]
    df7 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_7.csv'); df7 = df7.iloc[:,1:]
    df8 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_8.csv'); df8 = df8.iloc[:,1:]
    df9 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_9.csv'); df9 = df9.iloc[:,1:]
    df10 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_10.csv'); df10 = df10.iloc[:,1:]
    df11 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_11.csv'); df11 = df11.iloc[:,1:]
    df12 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_12.csv'); df12 = df12.iloc[:,1:]
    df13 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_13.csv'); df13 = df13.iloc[:,1:]
    df14 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_14.csv'); df14 = df14.iloc[:,1:]
    df15 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_15.csv'); df15 = df15.iloc[:,1:]
    df16 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_16.csv'); df16 = df16.iloc[:,1:]
    df17 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_17.csv'); df17 = df17.iloc[:,1:]
    df18 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_18.csv'); df18 = df18.iloc[:,1:]
    df19 = pd.read_csv('midas_NCa_Ca/midas_NCa_Ca_19.csv'); df19 = df19.iloc[:,1:]

    # Initialize the output dataframe with the same dimensions
    output_df = pd.DataFrame(index=df0.index, columns=df0.columns)

    # Process continuous columns
    for col in df1.columns[:35]:
        output_df[col] = (abs(df0[col]) + abs(df1[col]) + abs(df2[col]) + abs(df3[col]) + abs(df4[col]) + 
                          abs(df5[col]) + abs(df6[col]) + abs(df7[col]) + abs(df8[col]) + abs(df9[col]) + 
                         abs(df10[col]) + abs(df11[col]) + abs(df12[col]) + abs(df13[col]) + abs(df14[col]) + 
                          abs(df15[col]) + abs(df16[col]) + abs(df17[col]) + abs(df18[col]) + abs(df19[col])) / 20

    # Process categorical columns
    for col in df1.columns[35:]:
        output_df[col] = pd.concat([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
                                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=1).median(axis=1)
        
        # median = stats.median([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
        #                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=0).median[0]
        # output_df[col] = median

    # Write the output dataframe to a CSV file
    output_df.to_csv('output_NCa_Ca.csv', index=False)

    # re-order cols in output_df
    output_df_re = output_df[['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 
                              'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 
                              'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG', 'CACHEXSTAGE0VIG']].copy()

    return output_df_re





In [8]:
# calculate proportion of missingness in ndata_noR

# total num of NaN in the ndata_noR
total_nan_count = X_train.isna().sum().sum()

# total num of cells in ndata_noR
total_cells = X_train.size

# proportion of NaN
NaN_proportion = total_nan_count / total_cells

print('NaN_proportions:',NaN_proportion)


NaN_proportions: 0.014553014553014554


In [9]:
# 1.35% missingness

Xy_NCa_Ca_md = midas_impl(Xy_NCa_Ca, col_to_scale, 200)

Size index: [38, 2]

Computation graph constructed

Metal device set to: Apple M1 Pro
Model initialised

Epoch: 0 , loss: 105.70639190673828
Epoch: 1 , loss: 91.00410919189453
Epoch: 2 , loss: 89.78213348388672
Epoch: 3 , loss: 85.16680145263672
Epoch: 4 , loss: 78.87590789794922
Epoch: 5 , loss: 80.50151062011719
Epoch: 6 , loss: 85.22611999511719
Epoch: 7 , loss: 77.11125793457032
Epoch: 8 , loss: 81.9109603881836
Epoch: 9 , loss: 68.62443923950195
Epoch: 10 , loss: 69.00857391357422
Epoch: 11 , loss: 70.59335098266601
Epoch: 12 , loss: 66.72715606689454
Epoch: 13 , loss: 68.15931625366211
Epoch: 14 , loss: 66.77965621948242
Epoch: 15 , loss: 69.6884048461914
Epoch: 16 , loss: 61.745398712158206
Epoch: 17 , loss: 67.63983840942383
Epoch: 18 , loss: 65.35748062133788
Epoch: 19 , loss: 65.69388122558594
Epoch: 20 , loss: 59.95905990600586
Epoch: 21 , loss: 60.85670394897461
Epoch: 22 , loss: 61.98701782226563
Epoch: 23 , loss: 66.5122283935547
Epoch: 24 , loss: 64.04747619628907
Epoch:

In [10]:
Xy_NCa_Ca_md

,ENA.78,IFN.y,IL.10,IL.6,IL.8,MCP.1,MDC,MIP.1a,TNF.a,C.peptide,G.CSF,IL.22,Insulin,Leptin,MIP.3a,GRO.a,HGF,MMP.2,Adiponectin,CRP,GDF.15,TIMP.1,TGF.B2,TGF.B1,PPAR.y,HIF.1a,Laminin,HbA1c,CA19.9,Glucose,HDL,CCK,LDL,Triglyceride,Albumin,Lumican,ZAG,CACHEXSTAGE0VIG
0,0.973743,0.702204,0.170309,0.279777,0.642449,0.897066,0.847347,0.463522,0.504975,0.735082,0.597897,0.280568,0.370198,0.704687,0.329349,0.843594,0.689489,0.896706,0.944429,0.671947,0.648108,0.863826,0.809740,0.958342,0.557801,0.746461,0.855312,0.962120,0.301844,0.736581,0.697675,0.891682,0.813461,0.590579,0.992017,0.884276,0.933726,0.0
1,0.753619,0.673765,0.201269,0.216976,0.568140,0.897153,0.846711,0.490340,0.541123,0.805258,0.575445,0.603714,0.577541,0.716456,0.507407,0.763755,0.695094,0.851052,0.968519,0.654428,0.650953,0.852161,0.742037,0.936252,0.259342,0.864172,0.821863,0.865649,0.278378,0.823966,0.875018,0.896528,0.800485,0.680239,0.982816,0.934521,0.946027,1.0
2,0.845698,0.708108,0.589263,0.510682,0.668185,0.877033,0.857004,0.780488,0.689705,0.981980,0.485517,0.408600,0.914740,0.924848,0.422879,0.797677,0.698166,0.934024,0.939609,0.784915,0.709995,0.888243,0.396193,0.949125,0.456068,0.818775,0.794167,0.825031,0.995549,0.875911,0.948378,0.845673,0.831485,0.833340,0.986790,0.911570,0.938099,0.0
3,0.812632,0.347973,0.584275,0.685941,0.978330,0.896111,0.854521,0.597727,0.599266,0.837666,0.475654,0.011644,0.479707,0.539703,0.685997,0.878873,0.903304,0.859032,0.938919,1.000000,1.000000,1.000000,0.916020,0.925750,0.496821,0.859057,0.986485,0.891588,0.349025,0.915028,0.800792,0.943180,0.833575,0.891229,0.976420,0.909700,0.921943,1.0
4,0.874322,0.911611,0.545233,0.382143,0.560291,0.895160,0.873396,0.560855,0.623749,0.866990,0.667397,0.564354,0.671600,0.751458,0.513827,0.783358,0.695448,0.933030,0.920170,0.749777,0.743996,0.885778,0.665034,0.890141,0.239148,0.632491,0.928640,0.998678,0.594765,0.778181,0.750994,0.777503,0.838175,0.704706,0.990694,0.919573,0.917921,1.0
5,0.824705,0.672113,0.128262,0.394293,0.674223,0.840832,0.840238,0.506400,0.565337,0.865495,0.685015,0.237503,0.681181,0.874663,0.460349,0.738852,0.748400,0.856989,0.921624,0.833490,0.746252,0.889362,0.703698,0.915234,0.309573,0.774376,0.867157,0.805432,0.704206,0.828314,0.827834,0.786549,0.963486,0.681391,0.967102,0.925938,0.940252,1.0
6,0.841469,0.840877,0.480276,0.466352,0.790944,0.872012,0.883544,0.606755,0.671966,0.852460,0.558487,0.420953,0.634350,0.752996,0.601375,0.790870,0.806128,0.975402,0.990882,0.778754,0.877118,0.928648,0.750948,0.872878,0.371515,0.817630,0.912190,0.765249,0.883312,0.891417,0.884654,0.934632,0.929511,0.839016,0.973787,0.906189,0.921603,1.0
7,0.888476,0.934983,0.777184,0.513969,0.644025,0.892715,0.833911,0.602837,0.685216,0.730013,1.000000,0.714326,0.276333,0.727519,0.477759,0.766915,0.698471,0.927748,0.902796,0.766392,0.701257,0.834804,0.851352,0.896892,0.134704,0.955031,0.830003,0.775298,0.493626,0.775476,0.799242,0.859308,0.848570,0.859591,0.967249,0.957537,0.919996,1.0
8,0.822831,0.760066,0.341628,0.506309,0.818828,0.931089,0.897278,0.559788,0.744161,0.866087,0.612555,0.235436,0.697885,0.866286,0.447590,0.823908,0.799257,0.969826,0.874089,0.739207,0.727654,0.856589,0.823432,0.925430,0.133972,0.693184,0.856960,0.800062,0.790620,0.807665,0.765310,0.764378,0.801885,0.733022,0.975144,0.962794,0.925734,1.0
9,0.896836,0.483090,0.208708,0.332937,0.715294,0.874884,0.851394,0.664136,0.538878,0.731868,0.517583,0.121110,0.258069,0.717375,0.256370,0.763552,0.712563,0.871625,0.923336,0.787625,0.641227,0.869647,0.697929,0.926187,0.335863,0.719556,0.840633,0.896280,0.949400,0.782329,0.972487,0.828497,0.801383,0.668632,0.971846,0.866352,0.898846,0.0


In [11]:
# save to csv file

# 
Xy_NCa_Ca_md.to_csv('data/Xy_NCa_Ca_md.csv', index=False)